In [1]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


--


# Setup stuff: Connectivity

In [2]:

from katana import remote

my_client = remote.Client()

print(my_client)


In [3]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


--


In [4]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


<_Graph my_graph, 9y4Qb2faRRhDrFM1t7Z5HU4pDRg5Je9XwCuVkFPfTZX6, 3>


#  Actual analytics ..

In [ ]:

#  View the graph
#

l_result = my_graph.query(
   """
   MATCH (n) - [r] -> (m)
   RETURN n, r, m
   """ ,
   contextualize=True,
   )
l_result.view()


In [12]:

from katana.remote import analytics

print("--")


--


In [11]:

#  Getting the airport node id
#

def f_get_airport(i_arg1):
    
   l_query  = """
      MATCH (n: Airport) 
      WHERE n.airport_code = '{0}'
      RETURN ID(n) AS my_id
      """.format(l_airport)
   
   l_airport_id = my_graph.query(l_query)["my_id"][0]

   print("Airport id:  %s   %d" % (l_airport, l_airport_id))
  
   return (l_airport_id)


print("--")


--


In [14]:

#  Running analytics.ksssp() ..



l_airport1     = "MKE"
#  l_airport1     = "ORD"
#  l_airport1     = "DEN"
#  l_airport1     = "SJC"
   #
l_airport1_id  = f_get_airport(str(l_airport1))
   #
#  l_airport2     = "MKE"
#  l_airport2     = "ORD"
#  l_airport2     = "DEN"
l_airport2     = "SJC"
   #
l_airport2_id  = f_get_airport(str(l_airport2))


#  Below:  this currently fails on DISTANCE|NUM_HOPS.  Works with None.
#

#  Edge property
#
#  l_weight_property     = "DISTANCE"                 #  Edge property
#  l_weight_property     = "NUM_HOPS"
l_weight_property     = None   

l_max_iterations      = 10000 
l_kvalue              = 2


l_result = analytics.ksssp(
   my_graph,
      #
   start_node                    = l_airport1_id,
   target_node                   = l_source_nodeid,
      #
   edge_weight_property_name     = l_weight_property,
      #
   k_value                       = l_kvalue,        
      #
   max_iterations                = l_max_iterations,
   )

print("--")










          0/? [?op/s]

Airport id:  SJC   1


          0/? [?op/s]

Airport id:  SJC   1


InvalidRequestError: b'{"code":400,"message":"json: cannot unmarshal string into Go struct field KSSSPParameters.analytics.ksssp.start_node of type uint32"}'

In [8]:

#  analytics.sssp() adds a new property to the node
#

l_result = my_graph.query("""
   MATCH ( n ) 
   RETURN n
   
   """)

print(tabulate(l_result, headers='keys', tablefmt='psql'))

#  l_result = my_graph.query(
#     """
#     MATCH (n) - [r] -> (m)
#     RETURN r
#     """ )
#  print(tabulate(l_result, headers='keys', tablefmt='psql'))


          0/? [?op/s]

+----+-----------------+-------------+-----------+------------------+------------------+----------------+--------------------+----------+
|    |   n.internal_id | n.labels    | n.LABEL   | n.airport_code   | n.airport_name   |   n.ksssp_0021 |   n.sssp_dist_0021 | n.type   |
|----+-----------------+-------------+-----------+------------------+------------------+----------------+--------------------+----------|
|  0 |               0 | ['Airport'] | Airport   | ORD              | Chicago O-Hare   |              1 |                  1 | node     |
|  1 |               1 | ['Airport'] | Airport   | SJC              | San Jose         |              0 |                  0 | node     |
|  2 | 281474976710656 | ['Airport'] | Airport   | DEN              | Denver           |              1 |                  1 | node     |
|  3 | 562949953421312 | ['Airport'] | Airport   | MKE              | Milwaukee        |              2 |                  2 | node     |
+----+-----------------+----------

#  To remove a given property name from all nodes

In [ ]:

#  It is only required that you run this one time.
#
#  You may change the value i nthe text entry field as often as you wish.
#
from ipywidgets import widgets

my_column  = widgets.Text(
    value        = "sssp_dist_0021",
    placeholder  = "type here",
    description  = "Column Name: ",
    disabled     = False
)

my_column


In [ ]:

#  We could run the remove_property() directly, but then we couldn't cleanly catch
#  errors. So, run it inside a UDF.
#

def f_drop_column(i_graph, i_property):
   from katana import distributed
   try:
      i_graph.nodes.remove_property(i_property)
      return distributed.single_host(host=0, result=True)
   except LookupError as e:
      return distributed.single_host(host=0, result=False)
   except Exception as e:
      return distributed.single_host(host=0, result=False)
        

print("Deleting column name: %s" % (my_column.value))
   #
l_column = str(my_column.value)


if (my_graph.run(lambda g: f_drop_column(g, l_column))):
   print("Column ( %s ) deleted." % (my_column.value))
else:
   print("Column ( %s ) not found." % (my_column.value))



# Adding use of graph.project()

In [ ]:

#  graph.project() is used to create a homogenous group, from a heterogeneous one,
#  necessary to support some analytics
#
#  Here we'll add stuff to the graph that some analytics will not like ..


l_query = """
   CREATE ( n: Restaurant { restaurant_code: 'PAP' } )               //  Notice single curly braces
   SET n.restaurant_name = 'Pappadeauxs Seafood Kitchen' 
   """
      #
display(l_result1 = my_graph.query(l_query))

l_query = """
   MATCH
      (n: Airport   ),
      (m: Restaurant)
   WHERE n.airport_code     = 'SJC' AND m.restaurant_code  = 'PAP'
   CREATE (n) -[r: ATTRACTION { DRIVABLE: 'Yes' }]-> (m)
   """
      #
display(l_result2 = my_graph.query(l_query))


In [ ]:

l_result1 = my_graph.query(
   """
   MATCH (n) 
   RETURN n
   """ )

print(tabulate(l_result1, headers='keys', tablefmt='psql'))

l_result2 = my_graph.query(
   """
   MATCH (n) - [r] -> (m)
   RETURN r
   """ )

print(tabulate(l_result2, headers='keys', tablefmt='psql'))


# Run sssp() again, both with and without project()

In [ ]:

#  Running analytics.sssp() ..

#  From above
#
l_source_nodeid       = str(l_airport_id)


#  Below:  this currently fails on DISTANCE|NUM_HOPS.  Works with None.
#

#  Edge property
#
#  l_weight_property     = "DISTANCE"                 #  Edge property
#  l_weight_property     = "NUM_HOPS"
l_weight_property     = None   

l_max_iterations      = 10000 


#  Generate a unique prop name so we can run this over and over without edit ..
#
l_cntr += 1
   #
l_output_propname     = "sssp_dist_" + str(l_cntr).zfill(4)


analytics.sssp(
   my_graph,
      #
   source               = l_source_nodeid,
   weight_property      = l_weight_property,
   max_iterations       = l_max_iterations,
   result_property_name = l_output_propname,
   )

print("--")


In [ ]:
#  Check the output from routine above ..

l_result = my_graph.query(
   """
   MATCH (n) 
   RETURN n
   """ )

print(tabulate(l_result, headers='keys', tablefmt='psql'))

#  Sample output,
#
#     (nothing is output)


In [ ]:

#  Running analytics.sssp() should be run against a projection, a sub-graph ..

my_graph2 = my_graph.project(edge_types=["FLIES_TO"])


#  Running analytics.sssp() ..

#  From above
#
l_source_nodeid       = str(l_airport_id)


#  Below:  this currently fails on DISTANCE|NUM_HOPS.  Works with None.
#

#  Edge property
#
#  l_weight_property     = "DISTANCE"                 #  Edge property
#  l_weight_property     = "NUM_HOPS"
l_weight_property     = None   

l_max_iterations      = 10000 


#  Generate a unique prop name so we can run this over and over without edit ..
#
l_cntr += 1
   #
l_output_propname     = "sssp_dist_" + str(l_cntr).zfill(4)


analytics.sssp(
   my_graph2,
      #
   source               = l_source_nodeid,
   weight_property      = l_weight_property,
   max_iterations       = l_max_iterations,
   result_property_name = l_output_propname,
   )

print("--")


In [ ]:

#  Check the output from routine above ..

l_result = my_graph.query(
   """
   MATCH (n) 
   RETURN n
   """ )

print(tabulate(l_result, headers='keys', tablefmt='psql'))

